In [2]:
import unittest
import numpy as np
from nplm import NPLM



np.random.seed(0)

# Test settings
vocabulary_size = 5
context_size = 1
hidden_size = 1
feature_size = 2

# Gradient check
data_size = 4
inputs = np.random.randint(0, vocabulary_size, size=(data_size, context_size))
outputs = np.random.randint(0, vocabulary_size, size=(data_size, 1))
nplm = NPLM(vocabulary_size, feature_size, context_size, hidden_size)
result = nplm.gradient_check(inputs, outputs)

In [5]:
# Train NPLM
sentences = [['^', 'i', 'am'], ['^', 'you', 'are']]
nplm = NPLM(vocabulary_size, feature_size, context_size, hidden_size)
nplm.train(sentences, 100)

# Check if next word is predicted from context
for sentence in sentences:
    context = sentence[:-1]
    word = sentence[-1]
    prediction = nplm.predict(context)
    print('Prediction:{0} - True Val :{1}'.format(prediction,word))

Prediction:am - True Val :am
Prediction:are - True Val :are
